In [1]:
## This one is just to customise the appearnce of the dataframes , can be omitted
from IPython.display import display, Markdown
import numpy as np
def set_frame_style(df, caption="", font_size ='20px' ):
    random_list = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                      'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                      'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
    """Helper function to set dataframe presentation style.
    """
    return df.style.background_gradient(cmap=random_list[np.random.randint(1,17)]).set_caption(caption).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'Brown'),
        ('font-size', font_size),
        ('font-weight','bold')
    ]}])

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

- The python based google-play-scraper is currently failing to scrape reviews of any app on Google Play Store
- But it is possible to fetch 10k-20k reviews based on some tweaks, what some awesome people on Github came through.
- I'm sharing a woring example of how to do that based on the inputs from their Github Repository

In [2]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00


In [3]:
import google_play_scraper

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

### Get the app id of the Application from Playstore you want to fetch the reviews of.

- e.g. For Facebook app, the link on the Google Play Store is : https://play.google.com/store/apps/details/Facebook?id=com.facebook.katana&hl=en_ZA

- So the app id for Facebook is `com.facebook.katana`

In [4]:
app_id = 'com.facebook.katana'

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

### Some tweaks in the library files [Run as it is, no change needed in this]

In [5]:
from google_play_scraper import Sort
from google_play_scraper.constants.element import ElementSpecs
from google_play_scraper.constants.regex import Regex
from google_play_scraper.constants.request import Formats
from google_play_scraper.utils.request import post

import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time
import json
from time import sleep
from typing import List, Optional, Tuple

In [6]:


MAX_COUNT_EACH_FETCH = 199


class _ContinuationToken:
    __slots__ = (
        "token",
        "lang",
        "country",
        "sort",
        "count",
        "filter_score_with",
        "filter_device_with",
    )

    def __init__(
        self, token, lang, country, sort, count, filter_score_with, filter_device_with
    ):
        self.token = token
        self.lang = lang
        self.country = country
        self.sort = sort
        self.count = count
        self.filter_score_with = filter_score_with
        self.filter_device_with = filter_device_with


def _fetch_review_items(
    url: str,
    app_id: str,
    sort: int,
    count: int,
    filter_score_with: Optional[int],
    filter_device_with: Optional[int],
    pagination_token: Optional[str],
):
    dom = post(
        url,
        Formats.Reviews.build_body(
            app_id,
            sort,
            count,
            "null" if filter_score_with is None else filter_score_with,
            "null" if filter_device_with is None else filter_device_with,
            pagination_token,
        ),
        {"content-type": "application/x-www-form-urlencoded"},
    )
    match = json.loads(Regex.REVIEWS.findall(dom)[0])

    return json.loads(match[0][2])[0], json.loads(match[0][2])[-2][-1]


def reviews(
    app_id: str,
    lang: str = "en",
    country: str = "us",
    sort: Sort = Sort.MOST_RELEVANT,
    count: int = 100,
    filter_score_with: int = None,
    filter_device_with: int = None,
    continuation_token: _ContinuationToken = None,
) -> Tuple[List[dict], _ContinuationToken]:
    sort = sort.value

    if continuation_token is not None:
        token = continuation_token.token

        if token is None:
            return (
                [],
                continuation_token,
            )

        lang = continuation_token.lang
        country = continuation_token.country
        sort = continuation_token.sort
        count = continuation_token.count
        filter_score_with = continuation_token.filter_score_with
        filter_device_with = continuation_token.filter_device_with
    else:
        token = None

    url = Formats.Reviews.build(lang=lang, country=country)

    _fetch_count = count

    result = []

    while True:
        if _fetch_count == 0:
            break

        if _fetch_count > MAX_COUNT_EACH_FETCH:
            _fetch_count = MAX_COUNT_EACH_FETCH

        try:
            review_items, token = _fetch_review_items(
                url,
                app_id,
                sort,
                _fetch_count,
                filter_score_with,
                filter_device_with,
                token,
            )
        except (TypeError, IndexError):
            #funnan MOD start
            token = continuation_token.token
            continue
            #MOD end

        for review in review_items:
            result.append(
                {
                    k: spec.extract_content(review)
                    for k, spec in ElementSpecs.Review.items()
                }
            )

        _fetch_count = count - len(result)

        if isinstance(token, list):
            token = None
            break

    return (
        result,
        _ContinuationToken(
            token, lang, country, sort, count, filter_score_with, filter_device_with
        ),
    )


def reviews_all(app_id: str, sleep_milliseconds: int = 0, **kwargs) -> list:
    kwargs.pop("count", None)
    kwargs.pop("continuation_token", None)

    continuation_token = None

    result = []

    while True:
        _result, continuation_token = reviews(
            app_id,
            count=MAX_COUNT_EACH_FETCH,
            continuation_token=continuation_token,
            **kwargs
        )

        result += _result

        if continuation_token.token is None:
            break

        if sleep_milliseconds:
            sleep(sleep_milliseconds / 1000)

    return result

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

Set the number of reviews you want to scrape for

In [7]:
reviews_count = 25000

In [8]:
result = []
continuation_token = None


with tqdm(total=reviews_count, position=0, leave=True) as pbar:
    while len(result) < reviews_count:
        new_result, continuation_token = reviews(
            app_id,
            continuation_token=continuation_token,
            lang='en', #The language of review
            country='in', #Country for which you want to scrape 
            sort=Sort.MOST_RELEVANT,
            filter_score_with=None,
            count=199 #No need to change this
        )
        if not new_result:
            break
        result.extend(new_result)
        pbar.update(len(new_result))

25074it [01:04, 389.71it/s]


<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

And we scraped almost 25k reviews!

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

Sometimes the bar may stuck in between for some apps. It may mean that the number of reviews (not ratings) are almost the same as the iteration at which it stuck. Otherwise it may be the limitation of the library itself

<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">
    
Convert it to a Pandas Dataframe

In [9]:
df = pd.DataFrame(result)

set_frame_style(df.tail())

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
25069,222d4183-584a-4daf-a8d1-72c5f998a6db,Charlotte Hajek,https://play-lh.googleusercontent.com/a/ACg8ocI9Vdh4ADNnMTHFp35bcC8Dxt21NjlnRpWUWZ-Pw0Ij1Nu0nQ=mo,"Having problems with the app not working, not even loading and when it does work its forcing itself shut when I've been looking on the market or on my feed. I think it might have been since the last update. Having to keep uninstalling the updates and then updating it again over and over to get it to let me do anything on it.",3,1,274.0.0.46.119,2020-07-08 17:00:22,None,None,274.0.0.46.119
25070,4c0eba2d-7ce3-4d5b-8c52-b86eb93a36bf,Lelon Gilbert,https://play-lh.googleusercontent.com/a-/ALV-UjV8ZEZwVae-FzqdIv_WaV6setWlocYeLQoMD9-rrJjh8qAkXGEY,Overall it's a great app if use for a phew years even made new freinds but I loved scrolling through market place but it seems yall have did away with it I whould love to have market place back,2,8,446.0.0.27.352,2024-01-19 01:35:49,None,None,446.0.0.27.352
25071,b293c638-cfa4-470b-884f-38e243a63ad7,Lattra Slayton,https://play-lh.googleusercontent.com/a-/ALV-UjU-M5J4xsXSewprTD_EKTz7vb9fYLNZlC4DrUrlvpsiMaf3M6Qx,"It's a lot different and more difficult to use than when I first downloaded it several years ago! AND that go's for sharing and uploading videos too!! Thanks for allowing it to be free though!😁😉💋🦹‍♀️💃🏼 Now it also has a glitch and won't stay open ever since it made me sign in again to put my password in, it started doing it so I'm going to try and Uninstall it and reinstall it to see if that works...",3,219,380.0.0.29.109,2022-08-20 05:48:41,None,None,380.0.0.29.109
25072,f6c2353b-ce43-42f3-8e39-ea760294283b,Lizz L,https://play-lh.googleusercontent.com/a/ACg8ocJGNidw-m6Rwucgtx4Ru1SYbaJGx0icqK1ljee4xktQsREcRg=mo,Was working fine until marketplace stopped showing up I can't access it .,2,1,448.0.0.30.115,2024-02-01 03:18:00,None,None,448.0.0.30.115
25073,26773d4f-6acc-4786-af2c-2f104d716c95,nagaraj phy,https://play-lh.googleusercontent.com/a/ACg8ocK0kzURIXV5mqecDYAahDugZ1ssnZQ2yTTtDzENHJZrAmyoiA=mo,Like add per minute 10 times coming. It is like irritating. Please don't send me like adds.It is my humble request,2,1,440.0.0.31.105,2023-11-08 17:30:02,None,None,440.0.0.31.105


<div class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
   font-size : 15px;
    border-radius: 10px;
    font-color :  #581845  ;        
    border: 2px solid #581845;">

The Dataset is ready! Now, you can do whatever you want, from Sentiment Analysis to Facebook-specific NER to anything!